In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
import requests
import xmltodict

In [3]:
data = pd.read_csv('Starting Data.xlsx - Sheet1.csv')
ids = data["ID"][:-3]

In [4]:
ids

0       5046
1      27078
2      60015
3      63444
4      63445
       ...  
184    44842
185    44843
186    17225
187    31114
188    17234
Name: ID, Length: 189, dtype: object

In [6]:
xmls = {}
failed_ids = []

for id_ in tqdm(ids):
    try:
        url = f'https://papyri.info/hgv/{id_}/source'
        response = requests.get(url)
        xmls[id_] = xmltodict.parse(response.content)
    except:
        failed_ids += [id_]
        

100%|█████████████████████████████████████████| 189/189 [00:21<00:00,  8.78it/s]


In [7]:
xml_links = {}
xml_links['60015'] = 'https://raw.githubusercontent.com/papyri/idp.data/master/DCLP/61/60015.xml'
xml_links['63444'] = 'https://raw.githubusercontent.com/papyri/idp.data/master/DCLP/64/63444.xml'
xml_links['63445'] = 'https://raw.githubusercontent.com/papyri/idp.data/master/DCLP/64/63445.xml'
xml_links['63446'] = 'https://raw.githubusercontent.com/papyri/idp.data/master/DCLP/64/63446.xml'
xml_links['59208'] = 'https://raw.githubusercontent.com/papyri/idp.data/master/DCLP/60/59208.xml'
xml_links['64014'] = 'https://raw.githubusercontent.com/papyri/idp.data/master/DCLP/65/64014.xml'
xml_links['64015'] = 'https://raw.githubusercontent.com/papyri/idp.data/master/DCLP/65/64015.xml'
xml_links['64016'] = 'https://raw.githubusercontent.com/papyri/idp.data/master/DCLP/65/64016.xml'
xml_links['60444'] = 'https://raw.githubusercontent.com/papyri/idp.data/master/DCLP/61/60444.xml' 
xml_links['171914'] = 'https://raw.githubusercontent.com/papyri/idp.data/master/DCLP/172/171914.xml'
xml_links['44795'] = 'https://papyri.info/hgv/44795a/source'

In [8]:
for id_ in tqdm(list(xml_links.keys())):
    url = xml_links[id_]
    response = requests.get(url)
    xmls[id_] = xmltodict.parse(response.content)

100%|███████████████████████████████████████████| 11/11 [00:01<00:00,  6.41it/s]


In [26]:
duplicates = [942942, 942943, 942944, 942945, 942946, 942947]

In [27]:
no_redirects = [942948, 942949, 942950, 942951, 942953, 942956, 942957, 942958, 
               90781, 942952, 90780, 171911, 61914, 171909, 171910, 171912, 171913]

In [ ]:
multiple = [44795]

In [9]:
len(xmls)

166

In [45]:
import json 

with open("xmls.json", "w") as outfile:
    json.dump(xmls, outfile)

In [65]:
def get_content(item_id):
    return xmls[item_id]['TEI']['teiHeader']['fileDesc']['titleStmt']['title']

def get_img_url(item_id):
    divs = xmls[item_id]['TEI']['text']['body']['div']
    for d in divs:
        if d['@type'] == "figure":
            try:
                return d['p']['figure']['graphic']['@url']
            except:
                return d['p']['figure'][0]['graphic']['@url']
            
def get_material(item_id):
    return xmls[item_id]['TEI']['teiHeader']['fileDesc']['sourceDesc']['msDesc']['physDesc']['objectDesc']['supportDesc']['support']['material']

def get_origin(item_id):
    return xmls[item_id]['TEI']['teiHeader']['fileDesc']['sourceDesc']['msDesc']['history']['origin']['origPlace']


In [67]:
for item_id in xmls.keys():
    content = get_content(item_id)
    url = get_img_url(item_id)
    material = get_material(item_id)
    origin = get_origin(item_id)
    print(content,url,material,origin)

Fragmentary Contract http://wwwapp.cc.columbia.edu/ldpd/app/apis/item?mode=item&key=yale.apis.3100110000 Pergament Dura - Europos
Official Letter (?) http://wwwapp.cc.columbia.edu/ldpd/app/apis/item?mode=item&key=yale.apis.4400010000 Papyrus Dura - Europos
Deed of Sale http://wwwapp.cc.columbia.edu/ldpd/app/apis/item?mode=item&key=yale.apis.3100230000 Pergament Dura - Europos
Fragmentary Contract http://wwwapp.cc.columbia.edu/ldpd/app/apis/item?mode=item&key=yale.apis.4401200000 Papyrus Dura - Europos
Fragmentary List of Names (?) http://wwwapp.cc.columbia.edu/ldpd/app/apis/item?mode=item&key=yale.apis.4400860000 Papyrus Dura - Europos
Fragmentary Account http://wwwapp.cc.columbia.edu/ldpd/app/apis/item?mode=item&key=yale.apis.4400790000 Papyrus Dura - Europos
Greek Fragments http://wwwapp.cc.columbia.edu/ldpd/app/apis/item?mode=item&key=yale.apis.4400654200 Papyrus Dura - Europos
Greek Fragments http://wwwapp.cc.columbia.edu/ldpd/app/apis/item?mode=item&key=yale.apis.4401170000 Papyru

In [64]:
xmls[item_id]['TEI']['text']['body']['div'][2]['p']['figure'][0]['graphic']['@url']

'http://wwwapp.cc.columbia.edu/ldpd/app/apis/item?mode=item&key=yale.apis.4400970000'

## Old Code

In [5]:
data = pd.read_csv('Starting Data.xlsx - Sheet1.csv')
ids = data["ID"][:-3]

xmls = {}
failed_ids = []

for id_ in tqdm(ids):
    
    try:
    
        content = requests.get(f"https://papyri.info/trismegistos/{id_}").content
        soup = BeautifulSoup(content)
        url = soup.find("a",class_="xml")['href']

        if 'github' in url:
            url = url.replace('github.com', 'raw.githubusercontent.com')
            url = url.replace('/blob','')

        else:
            url = "https://papyri.info" + url

        response = requests.get(url)
        xmls[id_] = xmltodict.parse(response.content)
        
        
    except:
        
        failed_ids += [id_]
    

100%|█████████████████████████████████████████| 189/189 [00:59<00:00,  3.16it/s]


In [7]:
len(failed_ids)

104